In [1]:
from selenium import webdriver

In [72]:
from webdriver_manager.chrome import ChromeDriverManager
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import selenium
import json 
from selenium import webdriver
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import regex as re

In [3]:
import requests as req

from bs4 import BeautifulSoup as bs

In [4]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
opciones.add_argument("--no-sandbox")
opciones.add_argument("--disable-setuid-sandbox") 
opciones.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"

In [5]:
driver = "./chromedriver.exe"

In [6]:
PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Could not get version for google-chrome with the any command: google-chrome --version || google-chrome-stable --version
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Driver [/home/lapalmi87/.wdm/drivers/chromedriver/linux64/98.0.4758.80/chromedriver] found in cache


In [85]:
artists_id = pd.read_csv("Output_data/artists_spoti.csv",encoding = "ISO-8859-1")
artists_id.drop_duplicates(inplace=True)
artists_name = list(artists_id['name'])

In [82]:
%%time
url_try = 'https://www.songkick.com/search?page=1&per_page=10&type=upcoming&query=Adele'
html = req.get(url_try).content
soup = bs(html, 'html.parser')
soup.find_all("li" ,{"class":"concert event"})

datetime = re.findall('\d+-\d+-\d+\w+:\d+:\d+-\d+', str(soup.find("p", {"class":"date"})))
datetime

CPU times: user 422 ms, sys: 15.6 ms, total: 438 ms
Wall time: 656 ms


['2022-04-16T20:00:00-0700']

In [104]:
url_songkick = 'https://www.songkick.com/search?page=1&per_page=10&type=upcoming&query='
url_songkick2 = 'https://www.songkick.com/search?page=2&per_page=10&type=upcoming&query='
busqueda = artists_name

In [127]:
%%time
concerts = []
for i in artists_name:
    html = req.get(url_songkick+i).content
    soup = bs(html, 'html.parser')
    for e in soup.find_all("li" ,{"class":"concert event"}):
        # bucle para los conciertos
        location = e.find("p", {"class":"location"}).text.strip() # location
        date = e.find("p", {"class":"date"}).text.strip() # date
        concerts.append({
                'artist': i,
                'date': date,
                'location': location            
                })
    html2 = req.get(url_songkick2+i).content
    soup2 = bs(html2, 'html.parser')
    for e in soup2.find_all("li" ,{"class":"concert event"}):
        # bucle para los conciertos
        location = e.find("p", {"class":"location"}).text.strip() # location
        date = e.find("p", {"class":"date"}).text.strip() # date
        concerts.append({
                'artist': i,
                'date': date,
                'location': location          
               })

artist_concerts = pd.DataFrame(concerts)   

CPU times: user 8min 51s, sys: 12.2 s, total: 9min 3s
Wall time: 16min 50s


In [106]:
artist_concerts.drop_duplicates(inplace=True)
artist_concerts.shape

(6002, 3)

In [128]:
artist_concerts.reset_index(inplace=True)

In [129]:
artist_concerts.rename(columns={'index':'int_artist'},inplace=True)

In [131]:
artist_concerts.to_csv('/mnt/c/Users/Palmira/Desktop/Ironhack/Semana4/project_ETL/src/Output_data/upcoming_concerts.csv', index=False)

In [132]:
url_past = 'https://www.songkick.com/search?page=1&per_page=10&type=past&query='
url_past2 = 'https://www.songkick.com/search?page=2&per_page=10&type=past&query='

In [133]:
%%time
concerts_past = []
for i in artists_name:
    html = req.get(url_past+i).content
    soup = bs(html, 'html.parser')

    for e in soup.find_all("li" ,{"class":"concert event"}):
        # bucle para los conciertos
        location = e.find("p", {"class":"location"}).text.strip() # location
        date = e.find("p", {"class":"date"}).text.strip() # date
        
        concerts_past.append({
                'artist': i,
                'date': date,
                'location': location
                })
        
        
    html2 = req.get(url_past2+i).content
    soup2 = bs(html2, 'html.parser')
    for e in soup2.find_all("li" ,{"class":"concert event"}):
        # bucle para los conciertos
        location = e.find("p", {"class":"location"}).text.strip() # location
        date = e.find("p", {"class":"date"}).text.strip() # date
        
        concerts_past.append({
                'artist': i,
                'date': date,
                'location': location
                })

artist_concerts_past = pd.DataFrame(concerts_past)   

CPU times: user 13min 38s, sys: 16.3 s, total: 13min 55s
Wall time: 26min


In [134]:
artist_concerts_past.reset_index(inplace=True)
artist_concerts_past.rename(columns={'index':'int_artist'},inplace=True)

In [135]:
artist_concerts_past.sample(3)

,int_artist,artist,date,location
7499,7499,Armin van Buuren,Wednesday 22 January 2020,"House of Blues - Boston, Boston, MA, US"
9073,9073,Hillsong Worship,Thursday 01 November 2018,"Rolling Hills Community Church, Tualatin, OR, US"
10330,10330,Sfera Ebbasta,Saturday 28 April 2018,"Fabrique, Milan, Italy"


In [115]:
#artist_concerts_past.drop(axis=0, index=None, columns = ["level_0","index"], inplace=True)

In [136]:
artist_concerts_past.to_csv('/mnt/c/Users/Palmira/Desktop/Ironhack/Semana4/project_ETL/src/Output_data/past_concerts.csv', index=False)

artist_concerts_past['date'].str.extract('(?P<date>[\s\S]+) - (?P<message>[\s\S]+)', 
                                     expand=True)

artist_concerts_past[['date','day']] = artist_concerts_past['date'].str.split('\s+-\s+', expand=True)
for i in artist_concerts_past['date']:
    re.finall('[a-zA-Z]+', i)

## Aqui está lo de intentar entrar en spotify. 

#auth_manager=SpotifyClientCredentials(client_id="d663de0869444f16aa6d03d9c8ae4fe3",client_secret="47d3341ba76c4cf9a666c575b1c77ff9")

#url = 'https://open.spotify.com/artist/7MPGCB854Qo4alYMOPkBka'
#driver.get(url)
#mostrar_mas=driver.find_element_by_css_selector('#main > div > div.Root__top-container > div.Root__main-view > main > div.os-host.os-host-foreign.os-theme-spotify.os-host-resize-disabled.os-host-scrollbar-horizontal-hidden.main-view-container__scroll-node.os-host-transition.os-host-overflow.os-host-overflow-y > div.os-padding > div > div > div.main-view-container__scroll-node-child > section > div > div.iB16LxoPzDeVZo8zPhPQ > div:nth-child(3) > div:nth-child(1) > div > div.uhUnBNQXv7eaTTjCiPnp.m2I9Af9uHAIhYwi4fyo2 > div > div > div:nth-child(2) > div:nth-child(1) > div > div.bfQ2S9bMXr_kJjqEfcwA > div')
#mostrar_mas.click()

#driver.find_element_by_css_selector('//*[@id="main"]/div/div[2]/div[3]/main/div[2]/div[2]/div/div/div[2]/section/div/div[2]/div[3]/div[1]/div/div[1]/button/div').click

#driver.quit()

#url_songkick = 'https://api.songkick.com/api/3.0/search/artists.json?apikey={your API key}&query={artist}'

URL='https://www.google.com/search?q='

#busqueda='machine+learning+python'

driver=webdriver.Chrome(PATH)  # driver de selenium con chrome

driver.get(URL+busqueda)